In [ ]:
from pyspark import SparkContext
import matplotlib
#ALS is a matrix decomposition
from  pyspark.mllib.recommendation import ALS
sc = SparkContext("local[1]","My text")
data = sc.textFile("ml-100k/u.data")
print(data.first())

196	242	3	881250949


In [4]:
rawRatings = data.map(lambda record :record.split('\t')).map(lambda record:(record[0],record[1],record[2]))
print(rawRatings.first())

(u'196', u'242', u'3')


In [6]:
#this train is the Explicit train ,another way is the Implicit trainImplicit
model = ALS.train(rawRatings,10,10,0.1)
print(model)

In [7]:
model.predict(666,957)

2.6290556950040815

In [8]:
model.recommendProducts(666,5)

[Rating(user=666, product=1449, rating=4.761182699242415),
 Rating(user=666, product=1642, rating=4.537461076533329),
 Rating(user=666, product=1398, rating=4.466857155865764),
 Rating(user=666, product=1643, rating=4.4546740258403155),
 Rating(user=666, product=119, rating=4.3885679501144415)]

In [9]:
model.recommendUsers(957,5)

[Rating(user=810, product=957, rating=3.2371714530854945),
 Rating(user=770, product=957, rating=3.233474387376158),
 Rating(user=4, product=957, rating=3.1784173543805183),
 Rating(user=717, product=957, rating=3.1537911752350913),
 Rating(user=174, product=957, rating=3.1341053731860926)]

In [10]:
itemRDD = sc.textFile("ml-100k/u.item")
print(itemRDD.first())

1|Toy Story (1995)|01-Jan-1995||http://us.imdb.com/M/title-exact?Toy%20Story%20(1995)|0|0|0|1|1|1|0|0|0|0|0|0|0|0|0|0|0|0|0


In [11]:
movieTitle = itemRDD.map(lambda record:record.split('|')).map(lambda record:(int(record[0]),record[1])).collectAsMap()
print(movieTitle[2])

GoldenEye (1995)


In [12]:
movieIP = model.recommendProducts(666,5)
for ip in movieIP:
    print("the movie name is :"+movieTitle[ip[1]]+"and the score is :"+str(ip[2])+"\n")

the movie name is :Pather Panchali (1955)and the score is :4.76118269924

the movie name is :Some Mother's Son (1996)and the score is :4.53746107653

the movie name is :Anna (1996)and the score is :4.46685715587

the movie name is :Angel Baby (1995)and the score is :4.45467402584

the movie name is :Maya Lin: A Strong Clear Vision (1994)and the score is :4.38856795011



In [13]:
import numpy as np
#similar cosine
def Similarity(Vector1,Vector2):
    Vec1 = np.mat(Vector1)
    Vec2 = np.mat(Vector2)
    num = float(Vec1*Vec2.T)
    denom = np.linalg.norm(Vec1)*np.linalg.norm(Vec2)
    cos = num/denom
    return 0.5+0.5*cos
print(Similarity((1,1,0),(1,1,0)))
#Extraction vector factor,but I do not know how to get a specific vector
print(model.productFeatures)

0.9999999999999999
<bound method MatrixFactorizationModel.productFeatures of <pyspark.mllib.recommendation.MatrixFactorizationModel object at 0x7f9c8b6b1b10>>


In [14]:
ratings = data.map(lambda record :record.split('\t')).map(lambda record:(int(record[0]),int(record[1]),int(record[2]))).sortByKey(True,1)
print(ratings.take(5))

[(1, 61, 4), (1, 189, 3), (1, 33, 4), (1, 160, 4), (1, 20, 4)]


In [15]:
#select all the  movies which the 700 of users  has seen 
moviesForUser = ratings.filter(lambda record :record[0]==700)
print(moviesForUser.take(10))

[(700, 181, 5), (700, 144, 4), (700, 96, 4), (700, 50, 5), (700, 79, 3), (700, 180, 3), (700, 168, 3), (700, 28, 3), (700, 222, 3), (700, 531, 4)]


In [16]:
#select all the top 10 movies which the 700 of users  has seen 
topMovies = moviesForUser.takeOrdered(10,key=lambda record:-record[2])
print(topMovies)
print('----------------')
for movieIP in topMovies:
    print("the movie name is :"+movieTitle[movieIP[1]]+" and the score is "+str(movieIP[2])+"\n")

[(700, 181, 5), (700, 50, 5), (700, 173, 5), (700, 144, 4), (700, 96, 4), (700, 531, 4), (700, 318, 4), (700, 423, 4), (700, 174, 4), (700, 48, 4)]
----------------
the movie name is :Return of the Jedi (1983) and the score is 5

the movie name is :Star Wars (1977) and the score is 5

the movie name is :Princess Bride, The (1987) and the score is 5

the movie name is :Die Hard (1988) and the score is 4

the movie name is :Terminator 2: Judgment Day (1991) and the score is 4

the movie name is :Shine (1996) and the score is 4

the movie name is :Schindler's List (1993) and the score is 4

the movie name is :E.T. the Extra-Terrestrial (1982) and the score is 4

the movie name is :Raiders of the Lost Ark (1981) and the score is 4

the movie name is :Hoop Dreams (1994) and the score is 4



In [17]:
#To  test the accuracy of the model,we can use the  the 50 of moviesip to see the score that the model gives
#we can see the 700 user has given the movie 5 score, that is the movie is good
model.predict(700,50)

4.226843068482377

In [18]:
#Next , we use our model that has been trained to predict the top 10 movies of the 700 user likesP
predictMovieIP = model.recommendProducts(700,10)
print(predictMovieIP)
print('---------------------------')
for ip in predictMovieIP:
    print("the movie name is :"+movieTitle[ip[1]]+"and the score is :"+str(ip[2])+"\n")

[Rating(user=700, product=1463, rating=4.69470663539797), Rating(user=700, product=1643, rating=4.486657542068889), Rating(user=700, product=50, rating=4.226843068482377), Rating(user=700, product=1450, rating=4.086120598089442), Rating(user=700, product=173, rating=4.073710003642283), Rating(user=700, product=174, rating=4.072545286455303), Rating(user=700, product=172, rating=4.055783087719616), Rating(user=700, product=1467, rating=4.025754443087971), Rating(user=700, product=478, rating=3.92729328020497), Rating(user=700, product=181, rating=3.9155933878713594)]
---------------------------
the movie name is :Boys, Les (1997)and the score is :4.6947066354

the movie name is :Angel Baby (1995)and the score is :4.48665754207

the movie name is :Star Wars (1977)and the score is :4.22684306848

the movie name is :Golden Earrings (1947)and the score is :4.08612059809

the movie name is :Princess Bride, The (1987)and the score is :4.07371000364

the movie name is :Raiders of the Lost Ark 

In [19]:
#we will use "Mean Squared Error" (MSE) to evalute the model
#of course ,"the Root Mean Squared Error" can also been used.
#In fact ,there is no different !!!!!!!!!!!!!!!!!!!!!!!!
realRatings = []
predictRatings = []
for movieIP in topMovies:
    realRatings.append(float(movieIP[2]))
    predictRatings.append(model.predict(700,movieIP[1]))
print(realRatings)
print(predictRatings)
    

[5.0, 5.0, 5.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0]
[3.9155933878713594, 4.226843068482377, 4.073710003642283, 3.5384808736091125, 3.5871671252680026, 3.193776359091941, 3.7966400027336262, 3.532404597687263, 4.072545286455303, 3.345111956975095]


In [20]:
import math
def MSE(vect1,vect2):
    num = len(vect1)
    vect1 = np.mat(vect1)
    vect2 = np.mat(vect2)
    return np.sum(np.power((vect1-vect2),2))/num
resultMSE = MSE(realRatings,predictRatings)
resultRMSE = math.sqrt(resultMSE)
print("the MSE is :"+str(resultMSE))
print("the RMSE is :"+str(resultRMSE))

the MSE is :0.43592918604136444
the RMSE is :0.660249336267


In [21]:
#of course,we can use pyspark to make it!!!!!!!!!!!
realRatingsRDD = sc.parallelize(realRatings)
predictRatingsRDD = sc.parallelize(predictRatings)
predictAndRealRDD = predictRatingsRDD.zip(realRatingsRDD)
print(predictAndRealRDD.collect())

[(3.9155933878713594, 5.0), (4.226843068482377, 5.0), (4.073710003642283, 5.0), (3.5384808736091125, 4.0), (3.5871671252680026, 4.0), (3.193776359091941, 4.0), (3.7966400027336262, 4.0), (3.532404597687263, 4.0), (4.072545286455303, 4.0), (3.345111956975095, 4.0)]


In [22]:
resultMSE = predictAndRealRDD.map(lambda record:(record[0]-record[1])**2).mean()
resultRMSE =sc.parallelize([resultMSE]).map(lambda record :math.sqrt(record)).collect()
print("the MSE is :"+str(resultMSE))
print("the RMSE is :"+str(resultRMSE[0]))

the MSE is :0.435929186041
the RMSE is :0.660249336267


In [24]:
#There is also Build-in evalutation function in pyspark
#predictionAndLabel RDD of (predicted1 , label1) ,(predicted2 , label2),(predicted3 , label3),...
#so predicted.dimension must be equal to the label.dimension
from pyspark.mllib.evaluation import RegressionMetrics,RankingMetrics
metrics = RegressionMetrics(predictAndRealRDD)
print("the MSE is :"+str(metrics.meanSquaredError)+"\n")
print("the RMSE is :"+str(metrics.rootMeanSquaredError)+"\n")

the MSE is :0.435929186041

the RMSE is :0.660249336267



In [53]:
#Next,we can also use the "Mean Average Precision" (MAP) to calculate 
realmovies=moviesForUser.map(lambda record :record[1]).collect()
predictmovies = sc.parallelize(predictMovieIP).map(lambda record:record[1]).collect()
print(realmovies)
print(predictmovies)

[181, 144, 96, 50, 79, 180, 168, 28, 222, 531, 318, 423, 169, 73, 174, 173, 48, 202, 56, 651, 98]
[1463, 1643, 50, 1450, 173, 174, 172, 1467, 478, 181]


In [62]:
def MAP(pre,real,k):
    prek = sc.parallelize(pre).take(k)
    score = 0.0
    numHits = 0.0
    for (ip,movip) in  enumerate(prek):
        if movip in real:
            numHits+=1.0
            score+=numHits/float(ip+1.0)
    if len(real)==0:
        return 1.0
    else:
        return score/min(len(real),k)
resultMAP = MAP(predictmovies,realmovies,10)
print(resultMAP)

0.163333333333


In [ ]:
#Of course,there is also Build-in evalutation function in pyspark about MAP
#predictionAndLabel RDD of (predict Array ,label Array) pairs
#the dimension of predict does not have to be equal to the dimension of label

rankMetrics = RankingMetrics(predictmoviesAndRealmoviesRDD)
print("the MAP is:"+str(rankMetrics.meanAveragePrecision))